<a href="https://colab.research.google.com/github/HubertDomaros/ai-capstone-proj/blob/dev/pytorch_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
# prompt: import private dataset from https://www.kaggle.com/datasets/arct22/codebrim-original, use username and password from colab secrets

import os
from google.colab import userdata

# Replace with your actual Kaggle username and key
username = userdata.get('KAGGLE_USERNAME')
key = userdata.get('KAGGLE_KEY')

if username is None or key is None:
  raise ValueError("Please set KAGGLE_USERNAME and KAGGLE_KEY in Colab secrets.")

os.environ['KAGGLE_USERNAME'] = username
os.environ['KAGGLE_KEY'] = key

!pip install kaggle

# Download the dataset
!kaggle datasets download -d arct22/codebrim-original -p /content/datasets

!ls /content/datasets

# Unzip the downloaded dataset (assuming it is a zip file)
import zipfile
with zipfile.ZipFile('/content/datasets/codebrim-original.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/kaggle/input/codebrim-original')

!ls /content/kaggle/input/codebrim-original

Dataset URL: https://www.kaggle.com/datasets/arct22/codebrim-original
License(s): unknown
100% 7.73G/7.74G [01:56<00:00, 107MB/s]
100% 7.74G/7.74G [01:56<00:00, 71.3MB/s]
codebrim-original.zip
original_dataset


In [58]:
from src import annotation_extractor as an_ext
from src import image_processing as improc
from src import utils as u
from src import constants as c

import pandas as pd
import numpy as np
import cv2

In [48]:
dataset_path = os.path.join(os.getcwd(), r'kaggle/input/codebrim-original/original_dataset')
annotations_path = os.path.join(dataset_path, r'annotations')
annotations_df = an_ext.xml_annotations_to_dataframe(annotations_path)

In [55]:
annotations_df.sort_values(by='img', inplace=True)
annotations_df

,img,width,height,xmin,ymin,xmax,ymax,Background,Crack,Spallation,Efflorescence,ExposedBars,CorrosionStain
3832,image_0000005.jpg,1904,2856,661,472,992,1857,0,0,0,1,0,1
3833,image_0000005.jpg,1904,2856,1507,505,1904,2856,0,0,0,1,0,1
833,image_0000021.jpg,1904,2856,253,8,335,2108,0,0,0,1,0,0
3287,image_0000028.jpg,1752,2632,1173,1,1747,1077,0,0,0,1,0,0
3286,image_0000028.jpg,1752,2632,1,1,907,1043,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2099,image_0001599.jpg,4608,3456,3653,2813,4008,3365,0,0,1,0,1,1
2098,image_0001599.jpg,4608,3456,3339,734,3736,2826,0,0,0,0,0,1
2104,image_0001599.jpg,4608,3456,1002,1097,2931,3456,0,0,0,1,0,1
2101,image_0001599.jpg,4608,3456,2613,1,2776,408,0,0,1,0,1,1


In [59]:
def generate_images_with_metadata(input_file_path: str, output_folder: str, data: pd.DataFrame) -> dict:
    bboxes = data[c.bbox_coordinate_names].astype(int).to_numpy()
    multi_hot_encoded_labels = [list(x) for x in data[c.defect_names].astype(int).to_numpy()]

    augmented_images: list[improc.ImageAugumentor] = improc.generate_augmented_images(image_path=input_file_path,
                                                                                      bounding_boxes=bboxes,
                                                                                      label_values=multi_hot_encoded_labels,
                                                                                      resize=True, out_width=512,
                                                                                      out_height=512)

    out_dict_list = []
    for augmented_image in augmented_images:
        cv2.imwrite(os.path.join(output_folder, augmented_image.processed_image_name), augmented_image.processed_image)
        out_dict_list.append(augmented_image.metadata_dict)

    return u.unpack_lists(out_dict_list, out_dict_list[0].keys())

In [45]:
def process_image(input_folder, output_folder):
  output_list = []
  datas = 0
  aug_img_folder = r'kaggle/input/codebrim-original/original_dataset/augmented_imgs'
  for fname, data in annotations_df.groupby('img')[list(c.columns_list[1:])]:
      datas = data
      img_path = os.path.join(annotations_path, fname)

  improc.augment_image_and_save_to_folder(img_path, aug_img_folder, datas)


'/content/kaggle/input/codebrim-original-original-dataset/annotations'